In [ ]:
!pip install facenet-pytorch
!pip install Pillow
!pip install -q tensorflow==2.0.0
!pip install adversarial-robustness-toolbox[all]
!pip install matplotlib

In [ ]:
# LIBRERIE UTILI
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import numpy as np
from matplotlib import pyplot as plt
import art

if tf.__version__[0] != '2':
    raise ImportError('This notebook requires TensorFlow v2.')

print("GPU Available: ", tf.test.is_gpu_available())

In [ ]:
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from art.estimators.classification import PyTorchClassifier
resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True



fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath)

def load_image(filename):
    img = Image.open(filename)
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns

model = PyTorchClassifier(resnet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631) #This class implements a classifier with the PyTorch framework.

# **FSGM ATTACK**

In [ ]:
# Import the attack
from art.attacks.evasion import FastGradientMethod
from torch.nn import CrossEntropyLoss
from art.estimators.classification import PyTorchClassifier
from torchvision import transforms
from PIL import Image

def load_image(filename):
    img = Image.open(filename)
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns

model = PyTorchClassifier(resnet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631) #This class implements a classifier with the PyTorch framework.

test_img = load_image("test_set_cropped/Antonio_Cassano_8_face_0.jpg")

print(test_img.shape)
print(test_img.size)
test_img = test_img.unsqueeze(0)
print(test_img.shape)
print(test_img.size)
test_img = test_img.numpy()
print(test_img.shape)
print(type(test_img))

**NON TARGETED**

In [ ]:
#FSGM generic Attack for single sample

epsilon = 0.0005
attack = FastGradientMethod(estimator=model, eps=epsilon, targeted=False)
batch_size = test_img.shape[0]
targeted_labels = np.array([2] * batch_size)   #Al posto di 2 ci va l'indice della label della persona
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)   #Creazione attacco
test_images_adv = attack.generate(test_img)                                 # Generazione campione avversario
model_predictions = model.predict(test_images_adv)
loss = model.compute_loss(test_images_adv,one_hot_targeted_labels)                          # Predizione
perturbation = np.mean(np.abs((test_images_adv - test_img)))
predicted_label = LABELS[np.array(model_predictions[0].argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))


In [ ]:
import os
import re

dataset_dir = "test_set_cropped"

#FSGM generic Attack for all samples

correct_predictions = 0
total_images = 0
eps_range = [0.001, 0.005, 0.007, 0.01, 0.05, 0.07, 0.1, 0.5, 0.7]
accuracy_plot = []
for epsilon in eps_range:
    correct_predictions = 0
    total_images = 0
    attack = FastGradientMethod(estimator=model, eps=epsilon, targeted=False)

    for filename in os.listdir(dataset_dir):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            person_path = os.path.join(dataset_dir, filename)
            test_img = load_image(person_path)
            test_img = test_img.unsqueeze(0)
            test_img = test_img.numpy()
            test_images_adv = attack.generate(test_img)
            model_predictions = model.predict(test_images_adv)
            correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
            print("Etichetta corretta:", correct_label)   
            perturbation = np.mean(np.abs((test_images_adv - test_img)))
            predicted_label = LABELS[np.array(model_predictions[0].argmax())]
            print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
            total_images+=1
            
            predicted_label = str(predicted_label)

            if correct_label in predicted_label:
                correct_predictions+=1

            accuracy = correct_predictions/total_images
            print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

    if total_images != 0:
        final_accuracy = correct_predictions/total_images
        accuracy_plot.append(final_accuracy)
        print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
ax.plot(np.array(eps_range), np.array(accuracy_plot), 'b--', label='NN1')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('Accuracy')
plt.show()

NOTA: da inserire --> FSGM specific Attack for all samples

**TARGETED ATTACK**

In [ ]:
#FSGM specif Attack for single sample

target_class = 555
epsilon = 10000
attack = FastGradientMethod(estimator=model, eps=epsilon, targeted=True)

# Trasformazione etichetta categorica
targeted_labels = target_class*np.ones(LABELS.size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes = 8631)
test_images_adv = attack.generate(test_img, one_hot_targeted_labels)

#loss_test, accuracy_test = model.evaluate(test_images_adv, test_labels)
#print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
model_predictions = model.predict(test_images_adv)
perturbation = np.mean(np.abs((test_images_adv - test_img)))
print('Average perturbation: {:4.2f}'.format(perturbation))
#targeted_attack_loss, targeted_attack_accuracy = model.evaluate(test_images_adv, targeted_labels)
#print('Targeted attack accuracy: {:4.2f}'.format(targeted_attack_accuracy))
print("Etichetta target:{}".format(LABELS[target_class]))
print(model_predictions)
predicted_label = LABELS[np.array(model_predictions.argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))


# **PGD ATTACK**

https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/attacks/evasion.html#fast-gradient-method-fgm

In [ ]:
# Import attack
from torch.nn import CrossEntropyLoss
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier

#Impostare l'input shape
classifier = PyTorchClassifier(resnet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631) #This class implements a classifier with the PyTorch framework.


**NON-TARGETED ATTACK**

In [ ]:
epsilon = 0.1
eps_step = 0.1
max_iter = 1 

attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps = epsilon, eps_step=eps_step, targeted=False, max_iter = max_iter)

In [ ]:
# PGD generic Attack for single sample

test_images_adv = attack.generate(test_img)  # Utilizzare generate per generare i campion
model_predictions = classifier.predict(test_images_adv) # Classifier
predicted_label = LABELS[np.array(model_predictions.argmax())] 
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))

IDEA: Bloccare solo la massima epsilon e far variare solo le max_iter e gli eps_step.

In [ ]:
# PGD generic Attack for all samples
import os
import re

dataset_dir = "test_set_cropped_piccolo/" 
#eps_step = [0.0002, 0.0002, 0.0002, 0.001, 0.002, 0.002, 0.01, 0.02, 0.05] #La dimensione del passo della variazione dell'input a ogni iterazione. Questo parametro definisce quanto grande sarà ogni passo nella direzione del gradiente. #La dimensione del passo della variazione dell'input per il calcolo della perturbazione minima. Questo parametro è utilizzato quando minimal è impostato su True.
eps_step = [0.0002,0.002]
max_iter = [1,2,5,7]  # Il numero massimo di iterazioni. Questo parametro determina quante volte l'attacco iterativo sarà eseguito.
#eps_range = [0.001, 0.005, 0.007, 0.01, 0.05, 0.07, 0.1, 0.5, 0.7] #Questo valore determina l'ampiezza massima della perturbazione aggiunta agli input originali. Può essere un valore singolo (int o float) o un array numpy (ndarray).
eps_range = 0.7

accuracy_for_eps = []
accuracy_for_max_iter = []
perturbation_for_eps = []
perturbation_for_max_iter = []
correct_predictions = 0
total_images = 0
print("Inizio Attacco PGD NON-TARGETED")
for i in range(len(max_iter)):
        print("**** Attacco con max_iter {} **** ".format(max_iter[i]))
        for step in eps_step:   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps = eps_range, eps_step=step, targeted=False, max_iter = max_iter[i])
            
            print("**** Attacco con eps:{} con step a {} ****".format(eps_range,step))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image(person_path)
                    test_img = test_img.unsqueeze(0)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img)
                    model_predictions = model.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Salvo le perturbazioni applicate su ogni immagine
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1

                    accuracy = correct_predictions/total_images
                    print("Adversarial Sample misclassificati correttamente attuale: {}%".format((100-(accuracy*100))))
            

            if total_images != 0:    #Per ogni epsilon impostata salvo in accuracy plot
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images    #Calcolo la media delle perturbazioni applicate su tutte le immagini per una determinata epsilon
                    perturbation_for_eps.append(perturbazione_media)
                    print("----------- Perturbazione media aggiunta a tutte le immagini per eps_Step: {} equivale a {}% ----------------".format(step,perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps.append(final_accuracy)
                print("----------- Accuracy sugli adversarial Sample per eps_step: {} equivale a {}% ----------------".format(step,final_accuracy))

            #Per ogni epsilon impostata salvo in accuracy plot
        if True:#len(perturbation_for_eps) == len(eps_range):
            perturbazione_media_per_max_iter = sum(perturbation_for_eps)/len(eps_step) # Calcolo la perturbazione media dopo aver provato tutte le epsilon,quindi la media delle perturbazioni per eps_range
            perturbation_for_max_iter.append(perturbazione_media_per_max_iter)
            print("----------- Perturbazione media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],perturbation_for_max_iter[i]))
            perturbation_for_eps = []
        if True:#len(accuracy_for_eps) == len(eps_range):
            accuracy_media_per_max_iter = sum(accuracy_for_eps)/len(eps_step) # Calcolo l'accuracy media dopo aver provato tutte le epsilon,quindi la media dell'accuracy per ogni eps_range
            accuracy_for_max_iter.append(accuracy_media_per_max_iter)
            print("----------- Accuracy media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],accuracy_for_max_iter[i]))
            accuracy_for_eps = []
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")

# correct_predictions = 0
# total_images = 0
# for filename in os.listdir(dataset_dir):
#     if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#         person_path = os.path.join(dataset_dir, filename)
#         print("Immagine:", filename)
#         test_img = load_image(person_path)
#         test_img = test_img.unsqueeze(0)
#         test_img = test_img.numpy()
#         test_images_adv = attack.generate(test_img)
#         model_predictions = model.predict(test_images_adv)
#         correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
#         print("Etichetta corretta:", correct_label)   
#         perturbation = np.mean(np.abs((test_images_adv - test_img)))
#         predicted_label = LABELS[np.array(model_predictions[0].argmax())]
#         print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
#         total_images+=1

#         if predicted_label == correct_label:
#             correct_predictions+=1

#         accuracy = correct_predictions/total_images
#         print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

# if total_images != 0:
#     final_accuracy = correct_predictions/total_images
#     print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

**TARGETED ATTACK**

In [35]:
epsilon = 1
eps_step = 0.1
max_iter = 1 

attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps = epsilon, eps_step= eps_step, targeted=True, max_iter = max_iter)

In [ ]:
#PGD specific Attack for single sample

target_class = 10 

batch_size = test_img.shape[0]
targeted_labels = np.array([target_class] * batch_size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)

print(one_hot_targeted_labels.shape)
test_images_adv = attack.generate(test_img, one_hot_targeted_labels)


model_predictions = model.predict(test_images_adv)
perturbation = np.mean(np.abs((test_images_adv - test_img)))
print('Average perturbation: {:4.2f}'.format(perturbation))
#targeted_attack_loss, targeted_attack_accuracy = model.evaluate(test_images_adv, targeted_labels)
#print('Targeted attack accuracy: {:4.2f}'.format(targeted_attack_accuracy))
print("Etichetta target:{}".format(LABELS[target_class]))
print(model_predictions)
predicted_label = LABELS[np.array(model_predictions.argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))

In [36]:
#PGD specific Attack for all samples
import os
import re

dataset_dir = "test_set_cropped_piccolo/" 
#eps_step = [0.0002, 0.0002, 0.0002, 0.001, 0.002, 0.002, 0.01, 0.02, 0.05] #La dimensione del passo della variazione dell'input a ogni iterazione. Questo parametro definisce quanto grande sarà ogni passo nella direzione del gradiente. #La dimensione del passo della variazione dell'input per il calcolo della perturbazione minima. Questo parametro è utilizzato quando minimal è impostato su True.
eps_step = [0.0002,0.002]
max_iter = [1,2,5,7]  # Il numero massimo di iterazioni. Questo parametro determina quante volte l'attacco iterativo sarà eseguito.
#eps_range = [0.001, 0.005, 0.007, 0.01, 0.05, 0.07, 0.1, 0.5, 0.7] #Questo valore determina l'ampiezza massima della perturbazione aggiunta agli input originali. Può essere un valore singolo (int o float) o un array numpy (ndarray).
eps_range = 0.7

accuracy_misclassified_for_eps = []
accuracy_misclassified_for_max_iter = []
accuracy_for_eps = []
accuracy_for_max_iter = []
perturbation_for_eps = []
perturbation_for_max_iter = []
correct_misclassified = 0
total_images = 0

target_class = 10
etichetta_target = LABELS[0]
print("ETICHETTA TARGET: ", LABELS[10])
batch_size = test_img.shape[0]
targeted_labels = np.array([target_class] * batch_size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)

print("Inizio Attacco PGD NON-TARGETED")
for i in range(len(max_iter)):
        print("**** Attacco con max_iter {} **** ".format(max_iter[i]))
        for step in eps_step:   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps = eps_range, eps_step=step, targeted=True, max_iter = max_iter[i])
            
            print("**** Attacco con eps:{} con step a {} ****".format(eps_range,step))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image(person_path)
                    test_img = test_img.unsqueeze(0)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
                    model_predictions = model.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Salvo le perturbazioni applicate su ogni immagine
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1
                        
                    if etichetta_target in predicted_label:  
                        correct_misclassified = correct_misclassified+1   #Se il modello predice l'etichetta target allora è correttamente misclassificato

                    accuracy_misclassified = correct_misclassified/total_images
                    print("Adversarial Sample misclassificati correttamente attualmente: {}%".format((accuracy_misclassified)))
                    print("Accuracy attuale: {}%".format((correct_predictions/total_images)*100))
            

            if total_images != 0:    #Per ogni epsilon impostata salvo in accuracy plot
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images    #Calcolo la media delle perturbazioni applicate su tutte le immagini per una determinata epsilon
                    perturbation_for_eps.append(perturbazione_media)
                    print("----------- Perturbazione media aggiunta a tutte le immagini per eps_Step: {} equivale a {}% ----------------".format(step,perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps.append(final_accuracy)
                print("----------- Accuracy sugli adversarial Sample per eps_step: {} equivale a {}% ----------------".format(step,final_accuracy))
                
                accuracy_misclassified = correct_misclassified/total_images
                accuracy_misclassified_for_eps.append(accuracy_misclassified)
                print("----------- Adversarial Sample misclassificati correttamente: {}% -----------".format((accuracy_misclassified)))

            #Per ogni epsilon impostata salvo in accuracy plot
        if True:#len(perturbation_for_eps) == len(eps_range):
            perturbazione_media_per_max_iter = sum(perturbation_for_eps)/len(eps_step) # Calcolo la perturbazione media dopo aver provato tutte le epsilon,quindi la media delle perturbazioni per eps_range
            perturbation_for_max_iter.append(perturbazione_media_per_max_iter)
            print("----------- Perturbazione media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],perturbation_for_max_iter[i]))
            perturbation_for_eps = []
        if True:#len(accuracy_for_eps) == len(eps_range):
            accuracy_media_per_max_iter = sum(accuracy_for_eps)/len(eps_step) # Calcolo l'accuracy media dopo aver provato tutte le epsilon,quindi la media dell'accuracy per ogni eps_range
            accuracy_for_max_iter.append(accuracy_media_per_max_iter)
            print("----------- Accuracy media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],accuracy_for_max_iter[i]))
            accuracy_for_eps = []

        if True:
            accuracy__misclassified_media_per_max_iter = sum(accuracy_misclassified_for_eps)/len(eps_step) # Calcolo l'accuracy media dopo aver provato tutte le epsilon,quindi la media dell'accuracy per ogni eps_range
            accuracy_misclassified_for_max_iter.append(accuracy__misclassified_media_per_max_iter)
            print("----------- Accuracy MISCLASSIFIED media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],accuracy_misclassified_for_max_iter[i]))
            accuracy_misclassified_for_eps = []


        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")


# correct_predictions = 0
# total_images = 0
# target_class = 10
# print("ETICHETTA TARGET: ", LABELS[10])
# batch_size = test_img.shape[0]
# targeted_labels = np.array([target_class] * batch_size)
# one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)

# for filename in os.listdir(dataset_dir):
#     if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#         person_path = os.path.join(dataset_dir, filename)
#         print("Immagine:", filename)
#         test_img = load_image(person_path)
#         test_img = test_img.unsqueeze(0)
#         test_img = test_img.numpy()
#         test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
#         model_predictions = model.predict(test_images_adv)
#         correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
#         print("Etichetta corretta:", correct_label)   
#         perturbation = np.mean(np.abs((test_images_adv - test_img)))
#         predicted_label = LABELS[np.array(model_predictions[0].argmax())]
#         print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
#         total_images+=1

#         if predicted_label == correct_label:
#             correct_predictions+=1

#         accuracy = correct_predictions/total_images
#         print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

# if total_images != 0:
#     final_accuracy = correct_predictions/total_images
#     print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

ETICHETTA TARGET:   Aaron_Hernandez
Inizio Attacco PGD NON-TARGETED
**** Attacco con max_iter 1 **** 
**** Attacco con eps:0.7 con step a 0.0002 ****


Etichetta corretta: Andrea_Pirlo
Predetto  Andrea_Pirlo con probabilità 12.624496459960938 e con perturbazione 0.00019256384985055774
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 100.0%


Etichetta corretta: Antonio_Cassano
Predetto  Branch_Warren con probabilità 10.477907180786133 e con perturbazione 0.00019255444931332022
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 50.0%


Etichetta corretta: Ariana_Grande
Predetto  Ariana_Grande con probabilità 11.011987686157227 e con perturbazione 0.0001925574179040268
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 66.66666666666666%


Etichetta corretta: Ashley_Scott
Predetto  Ashley_Scott con probabilità 13.64526081085205 e con perturbazione 0.00019255466759204865
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 75.0%


...

Etichetta corretta: Boris_Johnson
Predetto  Vladimir_Petković con probabilità 8.49439525604248 e con perturbazione 0.008581671863794327
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 20.0%


Etichetta corretta: Brad_Pitt
Predetto  Sergio_Goyri con probabilità 8.982304573059082 e con perturbazione 0.008608371019363403
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 16.666666666666664%


Etichetta corretta: Christian_De_Sica
Predetto  Aaron_Hernandez con probabilità 10.330480575561523 e con perturbazione 0.008369944989681244
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 14.285714285714285%


Etichetta corretta: Christopher_Nolan
Predetto  Aaron_Hernandez con probabilità 12.09616470336914 e con perturbazione 0.008310322649776936
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 12.5%


Etichetta corretta: Cristiano_Ronaldo
Predetto  Aaron_Hernandez con probabilità 15.280435562133789 e con perturbazione 0.008631637319922447
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 11.11111111111111%
----------- Perturbazione media aggiunta a tutte le immagini per eps_Step: 0.002 equivale a 0.008549631159338687% ----------------
----------- Accuracy sugli adversarial Sample per eps_step: 0.002 equivale a 0.1111111111111111% ----------------
----------- Adversarial Sample misclassificati correttamente: 0.0% -----------
----------- Perturbazione media con iterazione 7 sugli adversarial Sample: 0.004881997456929336\% ----------------
----------- Accuracy media con iterazione 7 sugli adversarial Sample: 0.5\% ----------------
----------- Accuracy MISCLASSIFIED media con iterazione 7 sugli adversarial Sample: 0.0\% ----------------
---------------------------------------------------------------------------------------------------------------------

#  **CARLINI WAGNER **ATTACK****

In [ ]:
# Import all L-distance based attacks
from art.attacks.evasion import CarliniL2Method, CarliniL0Method, CarliniLInfMethod

Il metodo Carlini and Wagner L_0 Attack è un attacco iterativo che mira a trovare un esempio avversario minimizzando il numero di caratteristiche modificate (norma L_0). Ecco una spiegazione dettagliata del metodo e dei suoi parametri:

Metodo Carlini and Wagner L_0 Attack
Parametri del Metodo __init__
classifier (CLASSIFIER_CLASS_LOSS_GRADIENTS_TYPE):

Un classificatore addestrato che implementa i metodi per calcolare le perdite e i gradienti delle perdite rispetto agli input.
confidence (float = 0.0):

La fiducia degli esempi avversari: un valore più alto produce esempi che sono più lontani dall'input originale ma classificati con maggiore fiducia come la classe target. Questo parametro bilancia la necessità di modificare l'input originale con la certezza che l'output sia nella classe avversaria desiderata.
targeted (bool = False):

Indica se l'attacco è mirato a una specifica classe (True) o meno (False). In un attacco mirato, l'obiettivo è far classificare l'input avversario come una classe specifica scelta dall'attaccante. In un attacco non mirato, l'obiettivo è semplicemente far classificare l'input in una classe diversa da quella corretta.
learning_rate (float = 0.01):

Il tasso di apprendimento iniziale per l'algoritmo di attacco. Valori più piccoli producono risultati migliori ma convergono più lentamente.
binary_search_steps (int = 10):

Numero di volte in cui regolare la costante con la ricerca binaria (valore positivo). Se binary_search_steps è grande, l'algoritmo non è molto sensibile al valore di initial_const. Questo parametro controlla quante volte la costante di trade-off viene aggiustata per trovare il miglior equilibrio tra la distanza e la fiducia.
max_iter (int = 10):

Il numero massimo di iterazioni per l'attacco.
initial_const (float = 0.01):

La costante di trade-off iniziale c per regolare l'importanza relativa tra la distanza e la fiducia. Se binary_search_steps è grande, il valore iniziale di questa costante non è critico.
mask (ndarray | None = None):

Le caratteristiche iniziali che possono essere modificate dall'algoritmo. Se non specificato, l'algoritmo utilizza l'intero set di caratteristiche. Questo parametro può essere utilizzato per limitare l'attacco a modificare solo determinate parti dell'input.
warm_start (bool = True):

Invece di iniziare la discesa del gradiente in ogni iterazione dall'immagine iniziale, si inizia la discesa del gradiente dalla soluzione trovata nell'iterazione precedente. Questo parametro può accelerare la convergenza.
max_halving (int = 5):

Numero massimo di passi di dimezzamento nella ricerca lineare di ottimizzazione. Questo parametro controlla quante volte la dimensione del passo viene dimezzata durante la ricerca di un passo ottimale.
max_doubling (int = 5):

Numero massimo di passi di raddoppiamento nella ricerca lineare di ottimizzazione. Questo parametro controlla quante volte la dimensione del passo viene raddoppiata durante la ricerca di un passo ottimale.
batch_size (int = 1):

La dimensione del batch su cui vengono generati i campioni avversari. Questo parametro determina quanti campioni vengono processati contemporaneamente durante l'attacco.

verbose (bool = True):
Mostra barre di avanzamento. Se impostato su True, verranno visualizzati i progressi dell'attacco durante l'esecuzione.

**NON-TARGETED ATTACK**

In [ ]:
#Numero di volte in cui regolare la costante con la ricerca binaria (valore positivo). Se binary_search_steps è grande, l'algoritmo non è molto sensibile al valore di initial_const. Questo parametro controlla quante volte la costante di trade-off viene aggiustata per trovare il miglior equilibrio tra la distanza e la fiducia.
binary_search_steps = 1
#La fiducia degli esempi avversari: un valore più alto produce esempi che sono più lontani dall'input originale ma classificati con maggiore fiducia come la classe target. Questo parametro bilancia la necessità di modificare l'input originale con la certezza che l'output sia nella classe avversaria desiderata.
confidence = 0.5
warm_start = False #Invece di iniziare la discesa del gradiente in ogni iterazione dall'immagine iniziale, si inizia la discesa del gradiente dalla soluzione trovata nell'iterazione precedente. Questo parametro può accelerare la convergenza.
max_iter = 10
learning_rate = 0.01 #Il tasso di apprendimento iniziale per l'algoritmo di attacco. Valori più piccoli producono risultati migliori ma convergono più lentamente.
initial_const = 1000 #La costante di trade-off iniziale c per regolare l'importanza relativa tra la distanza e la fiducia. Se binary_search_steps è grande, il valore iniziale di questa costante non è critico.

attack = CarliniL2Method(classifier=classifier, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter, learning_rate=learning_rate, initial_const=initial_const, targeted=False)


In [ ]:
# Carlini Wagner generic attack on single sample

test_images_adv = attack.generate(test_img)

#loss_test, accuracy_test = model.evaluate(test_images_adv, test_labels)
model_predictions = model.predict(test_images_adv)
perturbation = np.mean(np.abs((test_images_adv - test_img)))
#print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))
print(LABELS[target_class])
print(model_predictions)
predicted_label = LABELS[np.array(model_predictions.argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))

DA ADATTARE ANCORA

In [ ]:
# Carlini Wagner generic attack on all samples
import os
import re

dataset_dir = "test_set_cropped_piccolo/" 
binary_search_steps = 1
confidence = 0.5
max_iter = 10
learning_rate = 0.01
initial_const = 1000

accuracy_for_eps = []
accuracy_for_max_iter = []
perturbation_for_eps = []
perturbation_for_max_iter = []
correct_predictions = 0
total_images = 0
print("Inizio Attacco PGD NON-TARGETED")
for i in range(len(max_iter)):
        print("**** Attacco con max_iter {} **** ".format(max_iter[i]))
        for step in eps_step:   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = CarliniL2Method(classifier=classifier, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter, learning_rate=learning_rate, initial_const=initial_const, targeted=False)
            
            print("**** Attacco con eps:{} con step a {} ****".format(eps_range,step))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image(person_path)
                    test_img = test_img.unsqueeze(0)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img)
                    model_predictions = model.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Salvo le perturbazioni applicate su ogni immagine
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1

                    accuracy = correct_predictions/total_images
                    print("Adversarial Sample misclassificati correttamente attuale: {}%".format((100-(accuracy*100))))
            

            if total_images != 0:    #Per ogni epsilon impostata salvo in accuracy plot
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images    #Calcolo la media delle perturbazioni applicate su tutte le immagini per una determinata epsilon
                    perturbation_for_eps.append(perturbazione_media)
                    print("----------- Perturbazione media aggiunta a tutte le immagini per eps_Step: {} equivale a {}% ----------------".format(step,perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps.append(final_accuracy)
                print("----------- Accuracy sugli adversarial Sample per eps_step: {} equivale a {}% ----------------".format(step,final_accuracy))

            #Per ogni epsilon impostata salvo in accuracy plot
        if True:#len(perturbation_for_eps) == len(eps_range):
            perturbazione_media_per_max_iter = sum(perturbation_for_eps)/len(eps_step) # Calcolo la perturbazione media dopo aver provato tutte le epsilon,quindi la media delle perturbazioni per eps_range
            perturbation_for_max_iter.append(perturbazione_media_per_max_iter)
            print("----------- Perturbazione media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],perturbation_for_max_iter[i]))
            perturbation_for_eps = []
        if True:#len(accuracy_for_eps) == len(eps_range):
            accuracy_media_per_max_iter = sum(accuracy_for_eps)/len(eps_step) # Calcolo l'accuracy media dopo aver provato tutte le epsilon,quindi la media dell'accuracy per ogni eps_range
            accuracy_for_max_iter.append(accuracy_media_per_max_iter)
            print("----------- Accuracy media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],accuracy_for_max_iter[i]))
            accuracy_for_eps = []
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")


# correct_predictions = 0
# total_images = 0
# for filename in os.listdir(dataset_dir):
#     if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#         person_path = os.path.join(dataset_dir, filename)
#         print("Immagine:", filename)
#         test_img = load_image(person_path)
#         test_img = test_img.unsqueeze(0)
#         test_img = test_img.numpy()
#         test_images_adv = attack.generate(test_img)
#         model_predictions = model.predict(test_images_adv)
#         correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
#         print("Etichetta corretta:", correct_label)   
#         perturbation = np.mean(np.abs((test_images_adv - test_img)))
#         predicted_label = LABELS[np.array(model_predictions[0].argmax())]
#         print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
#         total_images+=1

#         if predicted_label == correct_label:
#             correct_predictions+=1

#         accuracy = correct_predictions/total_images
#         print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

# if total_images != 0:
#     final_accuracy = correct_predictions/total_images
#     print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

**TARGETED ATTACK**


In [ ]:
binary_search_steps = 1
confidence = 0.5
max_iter = 10
learning_rate = 0.01
initial_const = 1000
target_class = 6

attack = CarliniL2Method(classifier=classifier, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter, learning_rate=learning_rate, initial_const=initial_const, targeted=True)


In [ ]:
# Carlini Wagner specific attack on single sample


# Trasformazione del dato categorico
targeted_labels = target_class*np.ones(LABELS.size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes = 8631)
test_images_adv = attack.generate(test_img, one_hot_targeted_labels)

model_predictions = model.predict(test_images_adv)
perturbation = np.mean(np.abs((test_images_adv - test_img)))
print('Average perturbation: {:4.2f}'.format(perturbation))
#targeted_attack_loss, targeted_attack_accuracy = model.evaluate(test_images_adv, targeted_labels)
#print('Targeted attack accuracy: {:4.2f}'.format(targeted_attack_accuracy))
print(LABELS[target_class])
print(model_predictions)
predicted_label = LABELS[np.array(model_predictions.argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))

In [ ]:
# Carlini Wagner specific attack on all samples


correct_predictions = 0
total_images = 0
target_class = 10

targeted_labels = target_class*np.ones(LABELS.size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes = 8631)
test_images_adv = attack.generate(test_img, one_hot_targeted_labels)

for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        person_path = os.path.join(dataset_dir, filename)
        print("Immagine:", filename)
        test_img = load_image(person_path)
        test_img = test_img.unsqueeze(0)
        test_img = test_img.numpy()
        test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
        model_predictions = model.predict(test_images_adv)
        correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
        print("Etichetta corretta:", correct_label)   
        perturbation = np.mean(np.abs((test_images_adv - test_img)))
        print('Average perturbation: {:4.2f}'.format(perturbation))
        predicted_label = LABELS[np.array(model_predictions[0].argmax())]
        print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
        
        total_images+=1

        if predicted_label == correct_label:
            correct_predictions+=1

        accuracy = correct_predictions/total_images
        print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

if total_images != 0:
    final_accuracy = correct_predictions/total_images
    print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

# CODICE PLOT IMMAGINI

In [ ]:
#Show one original example
plt.figure()
plt.matshow(test_images[0])
plt.title("Original Label: {}".format(test_labels[0]))
plt.show()

#Show the corresponding adversarial example
plt.figure()
plt.matshow(test_images_adv[0])
plt.title("Model Prediction: {}".format(np.argmax(model_predictions[0])))
plt.show()